# Imports

In [ ]:
from modules.data import  data_reader, data_loader
from modules.sr import result_saver
from modules.models import lag_llama
from modules.experiment.tscv import get_tscv_results, get_summary, extract_metrics
from modules.visualization import graphs
from modules.fine_tuning import lag_llama_ft
from modules.models import prpht


# Notes

Earliest monthly data for S&P500 is 1985
FTSE 100 also 1985
NASDAQ also 1985


In [15]:
from modules.data import data_loader

a = data_loader.get_data(type = "index", ticker = "NASDAQ Composite", frequency = "monthly", start = "1920-01-01", end = "2024-01-01", rtrn = False)
#a = data_loader.get_data(type = "stock", ticker = "IBM", frequency = "daily", start = "01-01-2023", end = "01-01-2024")

[*********************100%%**********************]  1 of 1 completed


In [17]:
x = data_loader.get_exogenous_data(start_date="1920-01-01", end_date="2024-01-01")

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:123: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values


In [18]:
x

,ds,x1,x2,x3
0,1986-01-01,0.274474,7.82,22.93
1,1986-02-01,-0.273723,7.69,15.46
2,1986-03-01,-0.457457,7.24,12.61
3,1986-04-01,-0.183824,6.60,12.84
4,1986-05-01,0.276243,6.65,15.38
...,...,...,...,...
451,2023-08-01,0.436716,5.44,81.39
452,2023-09-01,0.248513,5.49,89.43
453,2023-10-01,-0.038338,5.46,85.64
454,2023-11-01,-0.201514,5.41,77.69


In [16]:
a

,ds,y
0,1985-01-01,278.700012
1,1985-02-01,284.200012
2,1985-03-01,279.200012
3,1985-04-01,280.600006
4,1985-05-01,290.799988
...,...,...
463,2023-08-01,14034.969727
464,2023-09-01,13219.320312
465,2023-10-01,12851.240234
466,2023-11-01,14226.219727


In [ ]:
'''import os

# this is for renaming files

def rename_files_in_folder(folder_path, old_string, new_string):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            if old_string in filename:
                # Create the new filename by replacing the old string with the new string
                new_filename = filename.replace(old_string, new_string)
                
                # Construct the full file paths
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = os.path.join(folder_path, new_filename)
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                print(f"Renamed: {filename} -> {new_filename}")
            else:
                print(f"No match found in: {filename}")
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "evaluation"))

s1 = "TSCV_R"
s2 = "TSCVR"

rename_files_in_folder(folder_path, s1, s2)
rename_files_in_folder(folder_path_2, s1, s2)'''

In [39]:
'''# this is for adding something at the end

import os

# this is for renaming files

def rename_files_in_folder(folder_path):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = old_file_path[:-4] + "__BS=5__ME=4.csv"
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "evaluation"))



rename_files_in_folder(folder_path)
rename_files_in_folder(folder_path_2)'''

Renaming process completed.
Renaming process completed.


# Parameters

In [ ]:
# experiment parameters
#constraint: FOLDS x PREDICTION_LENGTH + TRAIN_SIZE !!must not!! be bigger than len(data)

PREDICTION_LENGTH = 1 # currentlz only works for PREDICTION_LENGTH > 1
TICKER = "S&P 500"
FREQUENCY = "minutely" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama", "autoregressor", "fine-tuned Llama"] # currentlz works onlz for these two
FOLDS = 5 # for TSCV # reduced to two for testing purposes
CONTEXT_LENGTH = 245 # set to 245 for testing purposes
METRICS = ['r2', 'mse', 'mae', 'rmse', 'mda', "mape"]

# fine-tuning parameters
BATCH_SIZE = 10
MAX_EPOCHS = 5

# data parameters
START_DATE = "2024-07-22"
END_DATE = "2024-07-23"

# want to add
#TRAIN_PERIOD = # context lenghts. Should take a look into this
TRAIN_SIZE = CONTEXT_LENGTH



# Data

In [ ]:
"""
data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
#if len(data) > 1:
data = data[0]

simple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])
#if len(simple_data) > 1:
simple_data = simple_data[0]

train_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])
train_data = train_data[0]

test_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])
test_data = test_data[0]
"""

In [ ]:
DATA_CONFIG = {"ticker" : TICKER,
               "frequency" : FREQUENCY,
               "start" : START_DATE,
               "end" : END_DATE}



In [ ]:
data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=DATA_CONFIG)

In [ ]:
data

In [ ]:
import 

# Additional parameters

In [ ]:
DATA_LENGTH = len(data)
FOLDS = int((DATA_LENGTH - TRAIN_SIZE) / PREDICTION_LENGTH) # this calculates max ammount of folds we can have given a set TRAIN_SIZE and PREDICTION_LENGTH
FOLDS = 5


In [ ]:
from modules.data import data_loader

In [ ]:
config = {"ticker" : "USD/GBP",
          "frequency" : "daily",
          "start" : "2024-07-01",
          "end" : "2024-07-30"}

In [ ]:
import os
print(os.getenv("ALPHAVANTAGE_API"))

In [ ]:
os.getenv("ALPHAVANTAGE_API")

In [ ]:
os.environ

In [ ]:
for key, value in os.environ.items():
    print(f'{key}: {value}')

In [ ]:
h = data_loader.get_data(data_type="exchange_rate", kwargs=config)

In [ ]:
h

# Fine tuning

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

In [ ]:
r, p = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY,
                           predictor=predictor)

# Saving results

In [ ]:
r

In [ ]:
p

In [ ]:
experiment_name = f"PREDICTION_LENGTH={PREDICTION_LENGTH}__TICKER={TICKER}__FREQUENCY={FREQUENCY}__TYPE_OF_DATA={TYPE_OF_DATA}__FOLDS={FOLDS}__CONTEXT_LENGTH/TRAIN_SIZE={CONTEXT_LENGTH}__FT_START_DATE={FT_START_DATE}__START_DATE={START_DATE}__END_DATE={END_DATE}__FT_LENGTH={FT_LENGTH}__DATA_LENGTH={DATA_LENGTH}" 

In [ ]:
experiment_name = f"P_L={PREDICTION_LENGTH}__T={TICKER}__FR={FREQUENCY}__T_O_D={TYPE_OF_DATA}__FO={FOLDS}__C_L_T_S={CONTEXT_LENGTH}__FT_S_D={FT_START_DATE}__S_D={START_DATE}__E_D={END_DATE}__FT_L={FT_LENGTH}__D_L={DATA_LENGTH}.csv"

In [ ]:
experiment_name

In [ ]:
result_saver.save_results(r, experiment_name, type="evaluation")
result_saver.save_results(p, experiment_name, type="prediction")

# Models

In [ ]:
# arima


In [ ]:
# lag llama

# Fine tuning an estimator

In [ ]:
PREDICTION_LENGTH = 5

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

# SKLearn TSCV

In [ ]:
r, p, a = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY, 
                           predictor=predictor)

In [ ]:
#creating the summary for each model
s = [get_summary(r[i]) for i in range(len(r))]

In [ ]:
means_df, medians_df, stds_df = extract_metrics(s, MODELS)

In [ ]:
means_df

# Visualisation

In [ ]:
graphs.standard_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.interactive_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.prediction_visualisation(MODELS, p, a)

LABEL the graph
label the axes with timestamps
show the history of the actual
table of comparison, not just graph